In [ ]:
%matplotlib inline
import os
import yt
import numpy as np
import matplotlib.pyplot as plt
import util

dirs = ['/home/ychen/data/0only_0529_h1',\
        '/home/ychen/data/0only_0605_hinf',\
        '/home/ychen/data/0only_0605_h0']
subdir = 'dtau_synchrotron_QU_nn_lobe/fits'

regex = 'synchrotron_i_lobe_0?00_150MHz.fits'

def rescan(dir, printlist=True):
    fitsdir = os.path.join(dir,subdir)
    files = util.scan_files(fitsdir, regex=regex, printlist=printlist, reverse=False)
    return files

fitsfiles = rescan(dirs[1])

In [ ]:
import pyfits

# Load the FITS file into the program
hdulist = pyfits.open(fitsfiles[0].fullpath)

# Load table data as tbdata
#tbdata = hdulist[1].data
hdulist[0].data.shape

In [ ]:
hdu = hdulist[0]
hdu.header

In [ ]:
def find_fwhm(arr_x, arr_y):

    difference = max(arr_y) - min(arr_y)
    HM = difference / 2

    pos_extremum = arr_y.argmax()  # or in your case: arr_y.argmin()

    nearest_above = (np.abs(arr_y[pos_extremum:-1] - HM)).argmin()
    nearest_below = (np.abs(arr_y[0:pos_extremum] - HM)).argmin()

    FWHM = (np.mean(arr_x[nearest_above + pos_extremum]) - 
        np.mean(arr_x[nearest_below]))
    return FWHM


In [ ]:
def plot_pol_histogram(frb_I, frb_Q, frb_U, factor=8):

    nx = frb_I.shape[1]//factor
    ny = frb_I.shape[0]//factor
    print(nx,ny)

    I_bin = frb_I.reshape(nx, factor, ny, factor).sum(3).sum(1)
    Q_bin = frb_Q.reshape(nx, factor, ny, factor).sum(3).sum(1)
    U_bin = frb_U.reshape(nx, factor, ny, factor).sum(3).sum(1)

    tot_bins = np.count_nonzero(I_bin.flatten())
    print('Total number of nonzero bins:', tot_bins)
    mask = I_bin > 1E-20
    #print mask
    print('Total number of masked bins:', sum(sum(mask)))

    # angle between the polarization and horizontal axis
    # (or angle between the magnetic field and vertical axis
    psi = 0.5*np.arctan2(U_bin, Q_bin)
    frac = np.sqrt(Q_bin**2+U_bin**2)/I_bin

    fig = plt.figure(figsize=(9,3))

    ax1 = fig.add_subplot(121)
    null = ax1.hist(frac[mask].flatten()*100, range=(0,80), bins=40)
    ax1.annotate('%s, %.2f Myr', % ('sim_name, '))
    ax1.set_xlabel('Polarization fraction (%)')
    ax1.set_xlim(0, 80)
    #ax1.set_ylim(0, 1200)
    #ax1.set_title(fitsfile.pathname.)

    ax2  = fig.add_subplot(122)
    n, bins, patches = ax2.hist(psi[mask].flatten(), 
                            bins=40, range=(-0.5*np.pi, 0.5*np.pi))
                            #weights=I_bin[mask].flatten())
    x_tick = np.linspace(-0.5, 0.5, 5, endpoint=True)
    ax2.set_xlabel('Position Angle')

    bin_center = (bins[1:]+bins[:-1])/2
    fwhm = find_fwhm(bin_center, n)
    ax2.annotate('FWHM=%.2f' % fwhm, (1,1), (0.7, 0.9), textcoords='axes fraction')


    x_label = [r"$-\pi/2$", r"$-\pi/4$", r"$0$", r"$+\pi/4$", r"$+\pi/2$"]
    ax2.set_xlim(-0.5*np.pi, 0.5*np.pi)
    #ax2.set_ylim(0,1600)
    ax2.set_xticks(x_tick*np.pi)
    ax2.set_xticklabels(x_label)
    #ax2.set_title(ds.basename + '  %.1f %s' % nu)

    #maindir = os.path.join(file.pathname, 'pol_synchrotron_QU_nn_%s/' % ptype)
    #histdir = os.path.join(maindir, 'histogram')
    #fig.savefig(histdir + '/' + ds.basename)
    plt.show()

In [ ]:
for dir in dirs:
    fitsfiles = rescan(dir, printlist=False)
    fitsfile = fitsfiles[-4]

    frb_I = pyfits.open(fitsfile.fullpath)[0].data
    frb_Q = pyfits.open(fitsfile.fullpath.replace('_i_', '_q_'))[0].data
    frb_U = pyfits.open(fitsfile.fullpath.replace('_i_', '_u_'))[0].data

    plot_pol_histogram(frb_I, frb_Q, frb_U)

In [ ]:
sum(n)

In [ ]:
factor = 4
nx = 2048/factor
ny = 4096/factor

I_bin = frb_I.reshape(nx, factor, ny, factor).sum(3).sum(1)
Q_bin = frb_Q.reshape(nx, factor, ny, factor).sum(3).sum(1)
U_bin = frb_U.reshape(nx, factor, ny, factor).sum(3).sum(1)

# angle between the polarization and horizontal axis
# (or angle between the magnetic field and vertical axis
psi = 0.5*np.arctan2(U_bin, Q_bin)
frac = np.sqrt(Q_bin**2+U_bin**2)/I_bin

fig = plt.figure(figsize=(16,4))

ax1 = fig.add_subplot(131)
null = ax1.hist(frac[I_bin.nonzero()].flatten()*100, range=(0,80), bins=40)
ax1.set_xlabel('Polarization fraction (%)')
ax1.set_xlim(0, 80)
#ax1.set_title(fitsfile.pathname.)

ax2  = fig.add_subplot(132)
n, bins, patches = ax2.hist(psi[I_bin.nonzero()].flatten(), 
                            bins=50, range=(-0.5*np.pi, 0.5*np.pi),
                            weights=I_bin[I_bin.nonzero()].flatten())
x_tick = np.linspace(-0.5, 0.5, 5, endpoint=True)
ax2.set_xlabel('Position Angle')

bin_center = (bins[1:]+bins[:-1])/2
fwhm = find_fwhm(bin_center, n)
ax2.annotate('FWHM=%.2f' % fwhm, (1,1), (0.7, 0.9), textcoords='axes fraction')


x_label = [r"$-\pi/2$", r"$-\pi/4$", r"$0$", r"$+\pi/4$", r"$+\pi/2$"]
ax2.set_xlim(-0.5*np.pi, 0.5*np.pi)
ax2.set_xticks(x_tick*np.pi)
ax2.set_xticklabels(x_label)
#ax2.set_title(ds.basename + '  %.1f %s' % nu)


ax3 = fig.add_subplot(133)
null = ax3.hist(np.abs(psi[I_bin.nonzero()].flatten()), bins=25, range=(0.0, 0.5*np.pi))
ax3.set_xlim(0.0, 0.5*np.pi)
ax3.set_xticks([x_tick[2:]*np.pi])
ax3.set_xticks(x_tick[2:]*np.pi)
ax3.set_xticklabels(x_label[2:])

#maindir = os.path.join(file.pathname, 'pol_synchrotron_QU_nn_%s/' % ptype)
#histdir = os.path.join(maindir, 'histogram')
#fig.savefig(histdir + '/' + ds.basename)
plt.show()

In [ ]:

plt.show()


In [ ]:
for dir in dirs:
    maindir = os.path.join(dir, 'pol_synchrotron_QU_nn_%s/' % ptype)
    histdir = os.path.join(maindir, 'histogram')
    for subdir in [maindir, histdir]:
        if not os.path.exists(subdir):
            os.mkdir(subdir)